<a href="https://colab.research.google.com/github/DatalakeLab/Pandas/blob/main/Pandas_POC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install "ipython>=7"

     |████████████████████████████████| 787 kB 26.0 MB/s 
     |████████████████████████████████| 370 kB 48.9 MB/s 
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting uninstall: ipython
    Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.20 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.27.0 which is incompatible.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.2 which is incompatible.
google-colab 1.0.0 requires request

In [1]:
!pip install dask[dataframe]

In [4]:
!pip install modin[ray]

     |████████████████████████████████| 715 kB 28.1 MB/s 
     |████████████████████████████████| 11.3 MB 28 kB/s 
     |████████████████████████████████| 49.6 MB 5.4 kB/s 
     |████████████████████████████████| 72 kB 629 kB/s 
     |████████████████████████████████| 127 kB 77.4 MB/s 
     |████████████████████████████████| 201 kB 72.5 MB/s 
     |████████████████████████████████| 1.3 MB 48.3 MB/s 
     |████████████████████████████████| 3.1 MB 74.4 MB/s 
     |████████████████████████████████| 65 kB 4.1 MB/s 
     |████████████████████████████████| 78 kB 7.2 MB/s 
     |████████████████████████████████| 85 kB 4.2 MB/s 
     |████████████████████████████████| 294 kB 52.6 MB/s 
     |████████████████████████████████| 142 kB 57.4 MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-py3-none-any.whl size=12617 sha256=9cc19da2c107f50bdb4c08f3d46e650a9c6efd6b1d063793e204fb27398c36f9
  Stored in directory: /root/.cache/pip/wheels/e6/67/af/f1ad15974b8fd95f59a63dbf854483ebe5c7a46a9393079

In [2]:
!pip install vaex


     |████████████████████████████████| 2.5 MB 25.7 MB/s 
     |████████████████████████████████| 42 kB 805 kB/s 
     |████████████████████████████████| 104 kB 54.4 MB/s 
     |████████████████████████████████| 952 kB 54.4 MB/s 
     |████████████████████████████████| 1.2 MB 48.8 MB/s 
     |████████████████████████████████| 2.9 MB 44.2 MB/s 
     |████████████████████████████████| 11.7 MB 40.0 MB/s 
     |████████████████████████████████| 3.3 MB 50.6 MB/s 
     |████████████████████████████████| 106 kB 73.4 MB/s 
     |████████████████████████████████| 260 kB 67.7 MB/s 
     |████████████████████████████████| 3.4 MB 54.1 MB/s 
     |████████████████████████████████| 2.7 MB 71.3 MB/s 
     |████████████████████████████████| 275 kB 73.2 MB/s 
     |████████████████████████████████| 52 kB 1.0 MB/s 
     |████████████████████████████████| 60 kB 7.1 MB/s 
     |████████████████████████████████| 10.1 MB 76.1 MB/s 
     |████████████████████████████████| 46 kB 3.6 MB/s 
     |██████████████

In [2]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 57 kB/s 
     |████████████████████████████████| 198 kB 54.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=a785e771cc15bd160f4acc822468a9d23e412834edf60f3b36791c97bcbfe6a5
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [2]:
import os
import sys
import gc
from time import time, sleep, strftime, localtime

import pandas as pd
import dask.dataframe as dd
import modin.pandas as mpd
import vaex
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, avg
# pandas on ray has moved to Modin
# import ray.dataframe as rpd

In [3]:
folder = "sample_data"
files = ["train_transaction.csv", "train_identity.csv"]
paths = [os.path.join(folder, f) for f in files]

In [4]:
stats = {}

In [5]:
pd.__version__

'1.3.2'

In [6]:
stats["pandas"] = {}
s = stats["pandas"]

ts = time()
df = pd.read_csv(paths[0])
te = time()
s["load_transactions"] = te-ts

ts = time()
df2 = pd.read_csv(paths[1])
te = time()
s["load_identity"] = te-ts

ts = time()
dff = df.merge(df2, on="TransactionID")
te = time()
s["merge"] = te-ts

ts = time()
grp = dff.groupby(["isFraud","ProductCD","card4","card6","id_15","id_31"])["TransactionAmt"].agg(["mean","sum"])
te = time()
s["aggregation"] = te-ts

ts = time()
dff.sort_values(by=["card1","addr1","D9"], inplace=True)
dff.sort_values(by=["addr1","D9","card1"], inplace=True)
dff.sort_values(by=["D9","card1","addr1"], inplace=True)
te = time()
s["sorting"] = te-ts

ts = time()
dff["card2"].value_counts()
te = time()
s["value_counts"] = te-ts

In [7]:
pd.DataFrame(stats)

,pandas
aggregation,0.034468
load_identity,0.539676
load_transactions,6.983413
merge,0.740309
sorting,0.395429
value_counts,0.001978


In [8]:
dff.to_pickle("sample_data/data/dff.pkl")

In [9]:
# Because julia groups by including N\A, let's just check that number of groups matches
grp = dff[["isFraud","ProductCD","card4","card6","id_15","id_31","TransactionAmt"]].fillna("~U~")\
.groupby(["isFraud","ProductCD","card4","card6","id_15","id_31"])["TransactionAmt"].agg(["mean","sum"])
grp.shape

(2459, 2)

In [10]:
def clean(wait_time: int=15):
    """Cleans created DataFrames and call the garbage collector to actions. Wait for 15s by default"""
    df, df2, dff, grp = None, None, None, None
    gc.collect()
    sleep(wait_time)
    return None

In [11]:
clean()

In [12]:
def list_variables_memory_usage() -> dict:
    """Memory of existing local variables"""
    local_vars = list(locals().items())
    return {var: sys.getsizeof(obj) for var, obj in local_vars}

**DASK**

In [13]:
stats["dask"] = {}
s = stats["dask"]

ts = time()
df = dd.read_csv(paths[0])
te = time()
s["load_transactions"] = te-ts

ts = time()
df2 = dd.read_csv(paths[1])
te = time()
s["load_identity"] = te-ts

ts = time()
dff = df.merge(df2, on="TransactionID")
te = time()
s["merge"] = te-ts

# the difference is that we call compute method, which runs all the computations at this point
ts = time()
grp = dff.groupby(["isFraud","ProductCD","card4","card6","id_15","id_31"])["TransactionAmt"]\
    .agg(["mean","sum"])\
    .compute()
te = time()
s["aggregation"] = te-ts

# parallel sorting is tricky that is why there are only work arounds in dask. 
ts = time()
dfs = dff.set_index("card1").compute()
te = time()
s["sorting"] = te-ts

In [14]:
type(dff), type(grp), type(dfs)

(dask.dataframe.core.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame)

In [15]:
clean()

In [16]:
pd.DataFrame(stats)

,pandas,dask
load_transactions,6.983413,0.093448
load_identity,0.539676,0.019784
merge,0.740309,0.046880
aggregation,0.034468,7.912935
sorting,0.395429,17.996132
value_counts,0.001978,NaN


In [18]:
stats["dask_indexed"] = {}
s = stats["dask_indexed"]

ts = time()
df = dd.read_csv(paths[0]).set_index("TransactionID")
te = time()
s["load_transactions"] = te-ts

ts = time()
df2 = dd.read_csv(paths[1]).set_index("TransactionID")
te = time()
s["load_identity"] = te-ts

ts = time()
dff = df.merge(df2, left_index=True, right_index=True)
te = time()
s["merge"] = te-ts

# the difference is that we call compute method, which runs all the computations at this point
ts = time()
grp = dff.groupby(["isFraud","ProductCD","card4","card6","id_15","id_31"])["TransactionAmt"]\
    .agg(["mean","sum"])\
    .compute()
te = time()
s["aggregation"] = te-ts

# parallel soring is tricky that is why there are only work arounds in dask. 
ts = time()
dff.set_index("card1").compute()
te = time()
s["sorting"] = te-ts

In [19]:
clean()
pd.DataFrame(stats)

,pandas,dask,dask_indexed
load_transactions,6.983413,0.093448,7.207852
load_identity,0.539676,0.019784,0.643515
merge,0.740309,0.046880,0.038707
aggregation,0.034468,7.912935,9.664391
sorting,0.395429,17.996132,22.026113
value_counts,0.001978,NaN,NaN


**Vaex**

In [20]:
vaex.__version__

{'vaex': '4.4.0',
 'vaex-core': '4.4.0',
 'vaex-viz': '0.5.0',
 'vaex-hdf5': '0.9.0',
 'vaex-server': '0.6.0',
 'vaex-astro': '0.8.3',
 'vaex-jupyter': '0.6.0',
 'vaex-ml': '0.13.0'}

In [21]:
tool = "vaex"
stats[tool] = {}
s = stats[tool]


ts = time()
#df = vaex.open(paths[0])
df = vaex.from_csv(paths[0])
te = time()
s["load_transactions"] = te-ts

ts = time()
df2 = vaex.open(paths[1])
te = time()
s["load_identity"] = te-ts

In [22]:
ts = time()
dff = df.join(df2, on="TransactionID")
te = time()
s["merge"] = te-ts

In [23]:
# the difference is that we call compute method, which runs all the computations at this point
ts = time()

grp = dff.groupby(["isFraud","ProductCD","card4","card6","id_15","id_31"], 
                  agg={'TransactionAmt':["mean","sum"]})

# alternatively
#grp = dff.groupby([dff["isFraud"],dff["ProductCD"],dff["card4"],dff["card6"],dff["id_15"],dff["id_31"]], 
#                  agg=[vaex.agg.mean('TransactionAmt'), vaex.agg.sum('TransactionAmt')])


te = time()
s["aggregation"] = te-ts

In [24]:
# the difference is that we call compute method, which runs all the computations at this point
ts = time()
dff_s = dff.sort(by=["card1","addr1","D9"])
dff_s = dff.sort(by=["addr1","D9","card1"])
dff_s = dff.sort(by=["D9","card1","addr1"])
te = time()
s["sorting"] = te-ts

In [25]:
ts = time()
dff["card2"].value_counts()
te = time()
s["value_counts"] = te-ts

In [26]:
pd.DataFrame(stats)

,pandas,dask,dask_indexed,vaex
load_transactions,6.983413,0.093448,7.207852,8.861584
load_identity,0.539676,0.019784,0.643515,0.957561
merge,0.740309,0.046880,0.038707,0.092737
aggregation,0.034468,7.912935,9.664391,0.751929
sorting,0.395429,17.996132,22.026113,0.247333
value_counts,0.001978,NaN,NaN,0.010245


In [27]:
clean()

PySpark

In [28]:
from pyspark import SparkContext
sc = SparkContext()
sc.version
sc.stop()

In [29]:
# Create my_spark
my_spark = SparkSession.builder \
    .master("local") \
    .appName("Pandas Alternative") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [30]:
tool = "spark"
stats[tool] = {}
s = stats[tool]


ts = time()
df = my_spark.read.csv(paths[0],inferSchema = True,header= True) 
te = time()
s["load_transactions"] = te-ts

ts = time()
df2 = my_spark.read.csv(paths[1],inferSchema = True,header= True) 
te = time()
s["load_identity"] = te-ts

In [31]:
ts = time()
dff = df.join(df2, "TransactionID")
te = time()
s["merge"] = te-ts

In [ ]:
# the difference is that we call collect method, which runs all the computations at this point
#ts = time()
#grp = dff.groupby([dff["isFraud"],dff["ProductCD"],dff["card4"],dff["card6"],dff["id_15"],dff["id_31"]]) \
#        .agg(avg("TransactionAmt"), sum("TransactionAmt"))\
#        .collect()
#te = time()
#s["aggregation"] = te-ts
#s["all"] = te-tss

In [32]:
# the difference is that we call collect method, which runs all the computations at this point
ts = time()
grp = dff.groupby(["isFraud","ProductCD","card4","card6","id_15","id_31"]) \
        .agg(avg("TransactionAmt"), sum("TransactionAmt"))\
        .collect()
te = time()
s["aggregation"] = te-ts

In [33]:
ts = time()
dff.orderBy("card1","addr1","D9").collect()
# alternatively
# dff.sort("card1","addr1","D9").collect()
te = time()
s["sorting"] = te-ts

In [34]:
ts = time()
dff.select("card2").distinct().collect()
te = time()
s["value_counts"] = te-ts

In [35]:
stats_df = pd.DataFrame(stats)
stats_df.loc['Total'] = stats_df.sum()
stats_df

,pandas,dask,dask_indexed,vaex,spark
load_transactions,6.983413,0.093448,7.207852,8.861584,19.301409
load_identity,0.539676,0.019784,0.643515,0.957561,1.490996
merge,0.740309,0.046880,0.038707,0.092737,0.304121
aggregation,0.034468,7.912935,9.664391,0.751929,12.733488
sorting,0.395429,17.996132,22.026113,0.247333,96.281583
value_counts,0.001978,NaN,NaN,0.010245,6.805352
Total,8.695272,26.069179,39.580577,10.921389,136.916949


**Modin**

In [36]:
mpd.__version__

'0.10.2'

In [37]:
tool = "modin"
stats[tool] = {}
s = stats[tool]


ts = time()
df = mpd.read_csv(paths[0])
te = time()
s["load_transactions"] = te-ts

ts = time()
df2 = mpd.read_csv(paths[1])
te = time()
s["load_identity"] = te-ts

ts = time()
dff = df.merge(df2, on="TransactionID")
te = time()
s["merge"] = te-ts

# modin defaults to pandas for multiple column aggregation and then fails on KeyError, though the key is available
ts = time()
try:
    grp = dff.groupby(["isFraud","ProductCD","card4","card6","id_15","id_31"])["TransactionAmt"].agg(["mean","sum"])
except Exception as e:
    print(e)
te = time()
s["aggregation"] = te-ts

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()

To request implementation, send an email to feature_requests@modin.org.


'isFraud'


In [38]:
pd.DataFrame(stats)

,pandas,dask,dask_indexed,vaex,spark,modin
load_transactions,6.983413,0.093448,7.207852,8.861584,19.301409,18.133374
load_identity,0.539676,0.019784,0.643515,0.957561,1.490996,0.766373
merge,0.740309,0.046880,0.038707,0.092737,0.304121,4.958086
aggregation,0.034468,7.912935,9.664391,0.751929,12.733488,0.310359
sorting,0.395429,17.996132,22.026113,0.247333,96.281583,NaN
value_counts,0.001978,NaN,NaN,0.010245,6.805352,NaN


In [39]:
clean()

In [40]:
pd.DataFrame(stats).to_csv("statistics.csv")

# **Performance test**

In [41]:
import psutil
import numpy as np

In [42]:
# use psutil to check for system resources
psutil.cpu_percent(), psutil.virtual_memory().percent

(28.8, 36.2)

In [43]:
def system_resources(n, pause, cpu_threshold = 0.5, mem_threshold = 0.5):
    
    cpu_m = []
    mem_m = []
    cpu_treshold_breached = False
    mem_treshold_breached = False
    
    for i in range(n):
        cpu_m.append(psutil.cpu_percent())
        mem_m.append(psutil.virtual_memory().percent)
        sleep(pause)
    cpu_m = np.mean(cpu_m)
    mem_m = np.mean(mem_m)
    
    if cpu_m / 100 > cpu_threshold:
        cpu_treshold_breached = True
        
    if mem_m / 100 > mem_threshold:
        mem_treshold_breached = True
    
    return {"cpu": cpu_m, "memory": mem_m, "cpu_threshold": cpu_treshold_breached, "mem_threshold": mem_treshold_breached }


In [44]:
system_resources(3, 1)

{'cpu': 8.433333333333334,
 'memory': 36.2,
 'cpu_threshold': False,
 'mem_threshold': False}

In [45]:
class Events:
    
    def __init__(self, path):
        self.file = open(path, 'a', encoding='utf-8')
        
    def log(self, time, tool, operation, duration):
        print("|".join([strftime('%Y-%m-%d %H:%M:%S', localtime(te)),tool,operation,str(duration)])+"\n")
        print(self.file)
        self.file.write("|".join([strftime('%Y-%m-%d %H:%M:%S', localtime(te)),tool,operation,str(duration)])+"\n")
        
    def close(self):
        self.file.close()

In [46]:
logger = Events("example.log")

In [47]:
ts = time()
df = pd.read_csv(paths[1])
te = time()
logger.log(te, "pandas", "load",  te-ts)

2021-08-30 00:46:18|pandas|load|0.681859016418457

<_io.TextIOWrapper name='example.log' mode='a' encoding='utf-8'>


In [48]:
logger.file.write("|".join([strftime('%Y-%m-%d %H:%M:%S', localtime(te)),"pandas","load",str(ts-te)])+"\n")

51

In [49]:
te = time()

In [51]:
import logging
logging.basicConfig(filename='sample_data/data/lexample.log', level=logging.DEBUG)


In [52]:
logging.info('This is an info message')